In [6]:
from ML.CNN.data_loader import load_data_qg_tagging
from keras import Sequential
from utilities import utilities
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
import keras

#### Loading the data 

In [7]:
X_train, y_train = load_data_qg_tagging(
    quark_data_path="../../Data/Trainning/q_jets.csv", gluon_data_path="../../Data/Trainning/g_jets.csv",
    eta_range=(-0.4, 0.4), phi_range=(-0.4, 0.4), n_eta_bins=16, n_phi_bins=16
)

X_val, y_val = load_data_qg_tagging(
    quark_data_path="../../Data/Validation/q_jets.csv", gluon_data_path="../../Data/Validation/g_jets.csv",
    eta_range=(-0.4, 0.4), phi_range=(-0.4, 0.4), n_eta_bins=16, n_phi_bins=16
)

#### Setting up the NN

In [17]:
regularization_rate = 10**(-8)
dropout_rate = 0.3

model = Sequential()
model.add(Input(shape=(16, 16, 1)))

model.add(Conv2D(filters = 128 , kernel_size=(5, 5), padding = "same", activation='relu',kernel_regularizer=l2(regularization_rate)))
model.add(MaxPool2D(pool_size=(2, 2),strides=(2, 2)))
model.add(Dropout(dropout_rate))

model.add(Conv2D(filters = 128 , kernel_size=(5, 5), padding = "same", activation='relu',kernel_regularizer=l2(regularization_rate)))
model.add(MaxPool2D(pool_size=(2, 2),strides=(2, 2)))
model.add(Dropout(dropout_rate))

model.add(Conv2D(filters = 64 , kernel_size=(3, 3), padding = "same", activation='relu',kernel_regularizer=l2(regularization_rate)))
model.add(MaxPool2D(pool_size=(2, 2),strides=(2, 2)))
model.add(Dropout(dropout_rate))

model.add(Flatten())

model.add(Dense(128, activation='relu', kernel_regularizer=l2(regularization_rate)))
model.add(Dropout(dropout_rate))

model.add(Dense(2, activation='softmax'))


learning_rate = 0.0001

optimizer = Adam(learning_rate=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy','auc'])

In [18]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 16, 16, 128)    │         3,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 8, 8, 128)      │       409,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 4, 4, 64)       │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 2, 2, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 2, 2, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 520,002 (1.98 MB)

 Trainable params: 520,002 (1.98 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
callbacks = [keras.callbacks.ModelCheckpoint(
        filepath = "model/QG_Tag_Model_3.keras",
        save_best_only=True,
        monitor = 'val_loss',
                    )]

history = model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=callbacks, batch_size=128)

Epoch 1/50
1964/1964 ━━━━━━━━━━━━━━━━━━━━ 216s 110ms/step - accuracy: 0.6895 - auc: 0.7389 - loss: 0.6034 - val_accuracy: 0.7322 - val_auc: 0.8023 - val_loss: 0.5498
Epoch 2/50
1964/1964 ━━━━━━━━━━━━━━━━━━━━ 226s 115ms/step - accuracy: 0.7231 - auc: 0.7874 - loss: 0.5583 - val_accuracy: 0.7455 - val_auc: 0.8217 - val_loss: 0.5256
Epoch 3/50
1964/1964 ━━━━━━━━━━━━━━━━━━━━ 266s 135ms/step - accuracy: 0.7395 - auc: 0.8127 - loss: 0.5306 - val_accuracy: 0.7535 - val_auc: 0.8331 - val_loss: 0.5059
Epoch 4/50
 865/1964 ━━━━━━━━━━━━━━━━━━━━ 3:34 196ms/step - accuracy: 0.7503 - auc: 0.8252 - loss: 0.5155

KeyboardInterrupt: 

In [ ]:
# perfoming the prediction
y_train_pred = model.predict(X_train)
y_val_pred = model.predict(X_val)

In [ ]:
print("Trainnig set:")
utilities.display_metrics(y_true=y_train[:, 0], y_pred=y_train_pred[:, 0] > 0.5)
print("Validation set:")
utilities.display_metrics(y_true=y_val[:, 0], y_pred=y_val_pred[:, 0] > 0.5)

In [ ]:
utilities.display_roc_curve(y_true=y_val[:, 0], y_score=y_val_pred[:, 0])

In [ ]:
utilities.plot_hist_trainning(history)

In [ ]:
utilities.save_model(model, history, "Quark_Gluon_Tagging_CNN")